# DLA with Multi processing 

This file contains a multi-threaded simulation for the DLA of the css_dla package.

### Imports and global variables

In [ ]:
import matplotlib.pyplot as plt
from css_dla import Model
import multiprocess as mp
import pickle as pkl
import numpy as np
import powerlaw

In [ ]:
N_MODELS = 30 # Number of models to be trained
N_PROCESS = 6 # Number of processes to use for running the models

# Model parameters
MODE = "multiple"
w = 600
h = 600
N_LOOPS = 10_000

# Create a filename to use for plots and pickled models
filename = f"{MODE}_{N_MODELS}_{N_LOOPS}its_{w}x{h}.pkl"

### Initializing the model

In [ ]:
models = []
for i in range(N_MODELS):
    models.append(Model(mode=MODE, w=w, h=h, seed=i))

### Running the model

In [ ]:
def run_model(mdl):
    mdl.loop(N_LOOPS)
    return mdl

pool = mp.Pool(processes=N_PROCESS)
models = pool.map(run_model, models)

In [ ]:
# Incorportate all variables into the filename
pkl.dump(models, open(f"data/{filename}", 'wb'))

### Plotting the results

In [ ]:
models = pkl.load(open(f"data/{filename}", 'rb'))

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(10, 10))
for i, mdl in enumerate(models):
    if i >= 6:
        break
    ax = axes[i//3, i%3]
    ax.imshow(mdl.grid, cmap='gray')
    ax.set_title(f'Model {i}')
    ax.axis('off')
plt.tight_layout()
print(np.sum(models[0].grid))

### Plot the density gradient

In [ ]:
# Plot the density gradient for six models. 
# Will still calculate the gradient for all models, for later use.
fig, axes = plt.subplots(2, 3, figsize=(10, 10))
dists = []
dens = []
for i, mdl in enumerate(models):
    distances, densities = mdl.density_gradient(1)
    dists.append(distances)
    dens.append(densities)
    
    if i < 6:
        ax = axes[i//3, i%3]
        ax.plot(distances, densities, '-0', color='blue')
plt.tight_layout()

In [ ]:


merged_dists = list(range(min([len(d) for d in dists])))
merged_densities = []
print(len(dists), len(merged_dists))
for i in range(len(merged_dists)):
    merged_densities.append([dens[j][i] / 2 for j in range(len(dens))])


def std(lst):
    return (sum([(x-np.average(lst))**2 for x in lst])/(len(lst)-1))**0.5

average_densities = np.average(merged_densities) 
confidence_intervals = 2 * std(merged_densities) / np.sqrt(len(merged_densities))

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot((merged_dists), np.log(average_densities), '-', color='blue')
ax.fill_between(merged_dists, np.log([average_densities[i]-confidence_intervals[i] for i in range(len(merged_dists))]), np.log([average_densities[i]+confidence_intervals[i] for i in range(len(merged_dists))]), color='blue', alpha=0.2)

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlabel('Distance from center (pixels)')
plt.ylabel('Density')

plt.savefig(f'density_gradient_{filename}.png', dpi=300)

In [ ]:
fit = powerlaw.Fit(average_densities, discrete=True)
print(fit.distribution_compare('power_law', 'exponential'))
print(fit.distribution_compare('power_law', 'truncated_power_law'))
print(fit.distribution_compare('power_law', 'lognormal'))

### Fractal Dimensions

In [ ]:
fractal_dims = []
for mdl in models:
    fractal_dims.append(mdl.get_fractal_dim())

In [ ]:
plt.figure(figsize=(2, 4))
plt.boxplot(fractal_dims,widths=[0.9])
plt.ylabel('Fractal dimension')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.xticks([])
plt.tight_layout()

plt.savefig(f'fractal_dimension_{filename}.png', dpi=300)
print(fractal_dims)